In [1]:
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
# Get SageMaker execution role
try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

In [3]:
role

'arn:aws:iam::041434534908:role/service-role/AmazonSageMaker-ExecutionRole-20250111T113739'

From the `role` output above, if we copy the section: `AmazonSageMaker-ExecutionRole-20250111T113739` and go to:
1. **AWS Home**
2. Open **IAM**
3. Go to **Roles**
4. Paste the role as `AmazonSageMaker-ExecutionRole-20250111T113739` in the search bar, and then click on it
5. We can see the execution roles we have given to this SageMaker profile

In [4]:
# Hub Model configuration. https://huggingface.co/models - TK: This is the model we are referring to from Hugging Face
hub = {
	'HF_MODEL_ID':'cardiffnlp/twitter-roberta-base-sentiment-latest',
	'HF_TASK':'text-classification'
}

In [5]:
# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	transformers_version='4.37.0',
	pytorch_version='2.1.0',
	py_version='py310',
	env=hub,
	role=role,
)

In [6]:
# deploy model to SageMaker Inference - TK: This will create a SageMaker endpoint
predictor = huggingface_model.deploy(
	initial_instance_count=1, # number of instances
	instance_type='ml.m5.xlarge' # ec2 instance type
)

------!

In [7]:
predictor.predict({
	"inputs": "I like you. I love you",
})

[{'label': 'positive', 'score': 0.9299361705780029}]

In [8]:
predictor.predict({"inputs": "The world is in shock today!"})

[{'label': 'negative', 'score': 0.8147163391113281}]

I will now go back to SageMaker -> Inference -> Endpoints & delete the Endpoint we just created.

This was just for testing purposes, we will fine-tune this model for our own dataset (News Headlines) which we will then deploy!